In [15]:
import pandas as pd
import numpy as np
import string
import nltk
from nltk.stem import WordNetLemmatizer
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize 
import re
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer

In [23]:
import nltk
nltk.download('punkt')

[nltk_data] Downloading package punkt to
[nltk_data]     /Users/linqisheng/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [89]:
disaster = pd.read_csv('nlp-getting-started/train.csv')

In [90]:
disaster.head()

,id,keyword,location,text,target
0,1,NaN,NaN,Our Deeds are the Reason of this #earthquake M...,1
1,4,NaN,NaN,Forest fire near La Ronge Sask. Canada,1
2,5,NaN,NaN,All residents asked to 'shelter in place' are ...,1
3,6,NaN,NaN,"13,000 people receive #wildfires evacuation or...",1
4,7,NaN,NaN,Just got sent this photo from Ruby #Alaska as ...,1


> ## add features


> ### cleaning

In [91]:
text_data = np.array(disaster[['id','text']])

In [92]:
lemmatizer = WordNetLemmatizer()
analyser = SentimentIntensityAnalyzer()
nltk.download('stopwords')
nltk.download('wordnet')
stop_words = set(stopwords.words('english')) 

[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/linqisheng/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     /Users/linqisheng/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [93]:
text_data[0:5,:]

array([[1,
        'Our Deeds are the Reason of this #earthquake May ALLAH Forgive us all'],
       [4, 'Forest fire near La Ronge Sask. Canada'],
       [5,
        "All residents asked to 'shelter in place' are being notified by officers. No other evacuation or shelter in place orders are expected"],
       [6,
        '13,000 people receive #wildfires evacuation orders in California '],
       [7,
        'Just got sent this photo from Ruby #Alaska as smoke from #wildfires pours into a school ']],
      dtype=object)

In [94]:
# Removes words that are not needed. Lemmaizes words
def clean_text(arr):
    cleaned_texts = []
    for row in arr:
        text = "".join((char for char in row[1] if char not in string.punctuation))
        text = re.sub(r'^https?:\/\/.*[\r\n]*', '', text, flags=re.MULTILINE)
        text = re.sub(r'^http?:\/\/.*[\r\n]*', '', row[1], flags=re.MULTILINE)
        text = word_tokenize(text)
        text = [lemmatizer.lemmatize(w.lower()) for w in text if not w in stop_words] 
        cleaned_texts.append(text)
        
    return np.array(cleaned_texts)

In [95]:
text_clean = clean_text(text_data)

In [96]:
disaster['clean_text'] = text_clean

> ### sentiment analysis

In [97]:
# Obtains sentiment score
def sentiment(arr):
    scores = []
    for row in arr:
        text = re.sub(r'^https?:\/\/.*[\r\n]*', '', row[1], flags=re.MULTILINE)
        text = re.sub(r'^http?:\/\/.*[\r\n]*', '', row[1], flags=re.MULTILINE)
        scores.append(list(analyser.polarity_scores(text).values()))
    return np.array(scores)

In [98]:
text_sentiment = sentiment(text_data)

In [99]:
positive = [t_s[0] for t_s in text_sentiment]
neutral = [t_s[1] for t_s in text_sentiment]
negative = [t_s[2] for t_s in text_sentiment]
compound = [t_s[3] for t_s in text_sentiment]

In [100]:
disaster['sentiment_score_positive'] = positive
disaster['sentiment_score_neutral'] = neutral
disaster['sentiment_score_negative'] = negative
disaster['sentiment_score_compound'] = compound

In [101]:
disaster.head()

,id,keyword,location,text,target,clean_text,sentiment_score_positive,sentiment_score_neutral,sentiment_score_negative,sentiment_score_compound
0,1,NaN,NaN,Our Deeds are the Reason of this #earthquake M...,1,"[our, deed, reason, #, earthquake, may, allah,...",0.000,0.851,0.149,0.2732
1,4,NaN,NaN,Forest fire near La Ronge Sask. Canada,1,"[forest, fire, near, la, ronge, sask, ., canada]",0.286,0.714,0.000,-0.3400
2,5,NaN,NaN,All residents asked to 'shelter in place' are ...,1,"[all, resident, asked, 'shelter, place, ', not...",0.095,0.905,0.000,-0.2960
3,6,NaN,NaN,"13,000 people receive #wildfires evacuation or...",1,"[13,000, people, receive, #, wildfire, evacuat...",0.000,1.000,0.000,0.0000
4,7,NaN,NaN,Just got sent this photo from Ruby #Alaska as ...,1,"[just, got, sent, photo, ruby, #, alaska, smok...",0.000,1.000,0.000,0.0000


In [102]:
# word_count
disaster['word_count'] = disaster['clean_text'].apply(lambda x: len(x))
# hashtag_count
disaster['hashtag_count'] = disaster['clean_text'].apply(lambda x: len([c for c in x if c == '#']))
#df_test['hashtag_count'] = df_test['text'].apply(lambda x: len([c for c in str(x) if c == '#']))

# mention_count
disaster['mention_count'] = disaster['clean_text'].apply(lambda x: len([c for c in x if c == '@']))
#df_test['mention_count'] = df_test['text'].apply(lambda x: len([c for c in str(x) if c == '@']))

In [142]:
disaster.head()

,id,keyword,location,text,target,clean_text,sentiment_score_positive,sentiment_score_neutral,sentiment_score_negative,sentiment_score_compound,word_count,hashtag_count,mention_count
0,1,NaN,NaN,Our Deeds are the Reason of this #earthquake M...,1,"[our, deed, reason, #, earthquake, may, allah,...",0.000,0.851,0.149,0.2732,9,1,0
1,4,NaN,NaN,Forest fire near La Ronge Sask. Canada,1,"[forest, fire, near, la, ronge, sask, ., canada]",0.286,0.714,0.000,-0.3400,8,0,0
2,5,NaN,NaN,All residents asked to 'shelter in place' are ...,1,"[all, resident, asked, 'shelter, place, ', not...",0.095,0.905,0.000,-0.2960,15,0,0
3,6,NaN,NaN,"13,000 people receive #wildfires evacuation or...",1,"[13,000, people, receive, #, wildfire, evacuat...",0.000,1.000,0.000,0.0000,8,1,0
4,7,NaN,NaN,Just got sent this photo from Ruby #Alaska as ...,1,"[just, got, sent, photo, ruby, #, alaska, smok...",0.000,1.000,0.000,0.0000,12,2,0


In [143]:
disaster.columns

Index(['id', 'keyword', 'location', 'text', 'target', 'clean_text',
       'sentiment_score_positive', 'sentiment_score_neutral',
       'sentiment_score_negative', 'sentiment_score_compound', 'word_count',
       'hashtag_count', 'mention_count'],
      dtype='object')

In [146]:
disaster = disaster[['sentiment_score_positive','sentiment_score_neutral',
       'sentiment_score_negative', 'sentiment_score_compound', 'word_count',
       'hashtag_count', 'mention_count', 'target']]

In [147]:
disaster.head()

,sentiment_score_positive,sentiment_score_neutral,sentiment_score_negative,sentiment_score_compound,word_count,hashtag_count,mention_count,target
0,0.000,0.851,0.149,0.2732,9,1,0,1
1,0.286,0.714,0.000,-0.3400,8,0,0,1
2,0.095,0.905,0.000,-0.2960,15,0,0,1
3,0.000,1.000,0.000,0.0000,8,1,0,1
4,0.000,1.000,0.000,0.0000,12,2,0,1


> # model

In [151]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.ensemble import AdaBoostClassifier, RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import RandomizedSearchCV, GridSearchCV
from sklearn.datasets import load_breast_cancer
from sklearn.metrics import f1_score

> ## baseline

In [148]:
X = np.array(disaster.drop('target', axis = 1))
y = np.array(disaster.target)

In [152]:
x_train, x_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

> ### Creates RandomizedSearch/ GridSearch CV objects

In [153]:
## Defines Dictionaries for possible values to choose from
ada_boost_dict = {'n_estimators': np.linspace(50, 500, num=46, dtype = int), 
                  'learning_rate':np.linspace(0.05,1,num=20), 
                  'algorithm' : ['SAMME', 'SAMME.R']}

log_dict = {'penalty': ['l2','none'], 
            'fit_intercept': [True, False],
           'C': np.linspace(0,5,21)}

rf_dict = {'n_estimators': np.linspace(50, 1000, num=96, dtype = int),
          'criterion':['gini','entropy'],
          'min_samples_split':np.linspace(5,50,11, dtype = int),
          'max_features':['auto','sqrt','log2',None],
          'bootstrap':[True, False]}

In [154]:
adaboost = AdaBoostClassifier()
rf = RandomForestClassifier(n_jobs=-1)
log_reg = LogisticRegression(multi_class = 'ovr', max_iter = 1000)

In [155]:
ada_model = RandomizedSearchCV(adaboost, ada_boost_dict, 60, 
                               random_state = 42, cv = 3)
rf_model = RandomizedSearchCV(rf, rf_dict, 300, 
                               random_state = 42, cv = 3)
log_model = GridSearchCV(log_reg, log_dict, cv = 3)

> ### Finds best parameters for each model

In [156]:
ada_model.fit(x_train, y_train)

RandomizedSearchCV(cv=3, error_score=nan,
                   estimator=AdaBoostClassifier(algorithm='SAMME.R',
                                                base_estimator=None,
                                                learning_rate=1.0,
                                                n_estimators=50,
                                                random_state=None),
                   iid='deprecated', n_iter=60, n_jobs=None,
                   param_distributions={'algorithm': ['SAMME', 'SAMME.R'],
                                        'learning_rate': array([0.05, 0.1 , 0.15, 0.2 , 0.25, 0.3 , 0.35, 0.4 , 0.45, 0.5 , 0.55,
       0.6 , 0.65, 0.7 , 0.75, 0.8 , 0.85, 0.9 , 0.95, 1.  ]),
                                        'n_estimators': array([ 50,  60,  70,  80,  90, 100, 110, 120, 130, 140, 150, 160, 170,
       180, 190, 200, 210, 220, 230, 240, 250, 260, 270, 280, 290, 300,
       310, 320, 330, 340, 350, 360, 370, 380, 390, 400, 410, 420, 430,
       440, 450, 46

In [157]:
rf_model.fit(x_train, y_train)

RandomizedSearchCV(cv=3, error_score=nan,
                   estimator=RandomForestClassifier(bootstrap=True,
                                                    ccp_alpha=0.0,
                                                    class_weight=None,
                                                    criterion='gini',
                                                    max_depth=None,
                                                    max_features='auto',
                                                    max_leaf_nodes=None,
                                                    max_samples=None,
                                                    min_impurity_decrease=0.0,
                                                    min_impurity_split=None,
                                                    min_samples_leaf=1,
                                                    min_samples_split=2,
                                                    min_weight_fraction_leaf=0.0,
               

In [158]:
log_model.fit(x_train, y_train)

/Users/linqisheng/opt/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/_logistic.py:935: RuntimeWarning: divide by zero encountered in double_scalars
  args=(X, target, 1. / C, sample_weight),
/Users/linqisheng/opt/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/_logistic.py:124: RuntimeWarning: invalid value encountered in double_scalars
  out = -np.sum(sample_weight * log_logistic(yz)) + .5 * alpha * np.dot(w, w)
/Users/linqisheng/opt/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/_logistic.py:129: RuntimeWarning: invalid value encountered in multiply
  grad[:n_features] = safe_sparse_dot(X.T, z0) + alpha * w
/Users/linqisheng/opt/anaconda3/lib/python3.7/site-packages/sklearn/model_selection/_validation.py:536: FitFailedWarning: Estimator fit failed. The score on this train-test partition for these parameters will be set to nan. Details: 
ValueError: Input contains NaN, infinity or a value too large for dtype('float64').

  FitFailedWarning)
/Users/li

/Users/linqisheng/opt/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/_logistic.py:1505: UserWarning: Setting penalty='none' will ignore the C and l1_ratio parameters
  "Setting penalty='none' will ignore the C and l1_ratio "
/Users/linqisheng/opt/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/_logistic.py:1505: UserWarning: Setting penalty='none' will ignore the C and l1_ratio parameters
  "Setting penalty='none' will ignore the C and l1_ratio "
/Users/linqisheng/opt/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/_logistic.py:1505: UserWarning: Setting penalty='none' will ignore the C and l1_ratio parameters
  "Setting penalty='none' will ignore the C and l1_ratio "
/Users/linqisheng/opt/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/_logistic.py:1505: UserWarning: Setting penalty='none' will ignore the C and l1_ratio parameters
  "Setting penalty='none' will ignore the C and l1_ratio "
/Users/linqisheng/opt/anaconda3/lib/python3.7/site-p

/Users/linqisheng/opt/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/_logistic.py:1505: UserWarning: Setting penalty='none' will ignore the C and l1_ratio parameters
  "Setting penalty='none' will ignore the C and l1_ratio "
/Users/linqisheng/opt/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/_logistic.py:1505: UserWarning: Setting penalty='none' will ignore the C and l1_ratio parameters
  "Setting penalty='none' will ignore the C and l1_ratio "
/Users/linqisheng/opt/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/_logistic.py:1505: UserWarning: Setting penalty='none' will ignore the C and l1_ratio parameters
  "Setting penalty='none' will ignore the C and l1_ratio "
/Users/linqisheng/opt/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/_logistic.py:1505: UserWarning: Setting penalty='none' will ignore the C and l1_ratio parameters
  "Setting penalty='none' will ignore the C and l1_ratio "
/Users/linqisheng/opt/anaconda3/lib/python3.7/site-p

/Users/linqisheng/opt/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/_logistic.py:1505: UserWarning: Setting penalty='none' will ignore the C and l1_ratio parameters
  "Setting penalty='none' will ignore the C and l1_ratio "
/Users/linqisheng/opt/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/_logistic.py:1505: UserWarning: Setting penalty='none' will ignore the C and l1_ratio parameters
  "Setting penalty='none' will ignore the C and l1_ratio "
/Users/linqisheng/opt/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/_logistic.py:1505: UserWarning: Setting penalty='none' will ignore the C and l1_ratio parameters
  "Setting penalty='none' will ignore the C and l1_ratio "
/Users/linqisheng/opt/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/_logistic.py:1505: UserWarning: Setting penalty='none' will ignore the C and l1_ratio parameters
  "Setting penalty='none' will ignore the C and l1_ratio "
/Users/linqisheng/opt/anaconda3/lib/python3.7/site-p

GridSearchCV(cv=3, error_score=nan,
             estimator=LogisticRegression(C=1.0, class_weight=None, dual=False,
                                          fit_intercept=True,
                                          intercept_scaling=1, l1_ratio=None,
                                          max_iter=1000, multi_class='ovr',
                                          n_jobs=None, penalty='l2',
                                          random_state=None, solver='lbfgs',
                                          tol=0.0001, verbose=0,
                                          warm_start=False),
             iid='deprecated', n_jobs=None,
             param_grid={'C': array([0.  , 0.25, 0.5 , 0.75, 1.  , 1.25, 1.5 , 1.75, 2.  , 2.25, 2.5 ,
       2.75, 3.  , 3.25, 3.5 , 3.75, 4.  , 4.25, 4.5 , 4.75, 5.  ]),
                         'fit_intercept': [True, False],
                         'penalty': ['l2', 'none']},
             pre_dispatch='2*n_jobs', refit=True, return_train_score=F

In [159]:
best_ada_params = ada_model.best_params_
best_log_params = log_model.best_params_
best_rf_params = rf_model.best_params_

> ### Determines the best model

In [160]:
best_log_reg = LogisticRegression(**best_log_params, multi_class = 'ovr', max_iter = 1000)
best_ada = AdaBoostClassifier(**best_ada_params)
best_rf = RandomForestClassifier(**best_rf_params, n_jobs=-1)

In [161]:
best_rf.fit(x_train, y_train)
best_log_reg.fit(x_train, y_train)
best_ada.fit(x_train, y_train)

AdaBoostClassifier(algorithm='SAMME.R', base_estimator=None, learning_rate=0.6,
                   n_estimators=280, random_state=None)

In [162]:
y_pred_rf = best_rf.predict(x_test)
y_pred_ada = best_ada.predict(x_test)
y_pred_log_reg = best_log_reg.predict(x_test)

In [163]:
f1_score(y_pred_rf, y_test)

0.5552654482158399

In [164]:
f1_score(y_pred_ada, y_test)

0.5343777197563099

In [165]:
f1_score(y_pred_log_reg, y_test)

0.5237258347978909

> ### Add N-grams (For later)

In [105]:
disaster_t = disaster.loc[disaster['target']==1]

In [107]:
n_disaster_t = disaster.loc[disaster['target']==0]

In [123]:
from collections import defaultdict

def generate_ngrams(text, n_gram=2):
    token = [token for token in text.lower().split(' ') if token != '' if token not in stop_words]
    ngrams = zip(*[token[i:] for i in range(n_gram)])
    return [' '.join(ngram) for ngram in ngrams]

N = 100
# Bigrams
disaster_bigrams = defaultdict(int)
nondisaster_bigrams = defaultdict(int)

for tweet in disaster_t['text']:
    for word in generate_ngrams(tweet, n_gram=2):
        disaster_bigrams[word] += 1
        
for tweet in n_disaster_t['text']:
    for word in generate_ngrams(tweet, n_gram=2):
        nondisaster_bigrams[word] += 1
        
df_disaster_bigrams = pd.DataFrame(sorted(disaster_bigrams.items(), key=lambda x: x[1])[::-1])
df_nondisaster_bigrams = pd.DataFrame(sorted(nondisaster_bigrams.items(), key=lambda x: x[1])[::-1])

In [126]:
df_disaster_bigrams.columns = ['combination', 'count']

In [134]:
bigram_list_combination = [comb.split(' ') for comb in df_disaster_bigrams['combination']]

In [137]:
disaster['clean_text'][0] in bigram_list_combination

False

In [141]:
len(bigram_list_combination)

24591

In [ ]:
disaster['clean_text'][0] in bigram_list_combination

False

In [113]:
df_nondisaster_bigrams

,0,1
0,- full,60
1,cross body,38
2,@youtube video,34
3,liked @youtube,34
4,full reû_,25
...,...,...
33297,london cool,1
33298,car fast,1
33299,summer lovely,1
33300,love fruits,1


In [116]:
df_nondisaster_bigrams.columns = [['combination', 'count']]

In [25]:
[lemmatizer.lemmatize(w.lower()) for w in word_tokenize(test_string) if not w in stop_words] 

['our', 'deed', 'reason', 'earthquake', 'may', 'allah', 'forgive', 'u']